In [199]:
%matplotlib inline
from bs4 import BeautifulSoup
from pymongo import MongoClient
from bson import ObjectId
import requests
import datetime
import ssl
import re
import time
import json
import random
import pprint

In [ ]:
ID_PATTERN = re.compile("id=(\d+)")

In [2]:
ssl._create_default_https_context = ssl._create_unverified_context  # 全局取消网页安全验证

In [3]:
client = MongoClient('localhost', 27017)
db = client.moniter_crowdfunding
db.authenticate(name="worker", password="hard", mechanism="SCRAM-SHA-1")
project = db.projects
s_project = db.success_projects
f_project = db.failure_projects
p_founder = db.founders

In [82]:
p_founder.find_one()

{'_id': '100006',
 '支持项目': {},
 '关注项目': {'100006': {'prj_name': '升级保健发热腰带 健康撑腰',
   'prj_desc': '自主品牌的升级型保健电热腰带秉承着“用心造，健康美”的理念研发及生产。我们为健康而来，\r\n致力于用随意的方式带给您和您家人健康的生活。',
   'prj_href': '/project/details/100006.html'}},
 '发起项目': {'100006': {'prj_name': '升级保健发热腰带 健康撑腰',
   'prj_desc': '自主品牌的升级型保健电热腰带秉承着“用心造，健康美”的理念研发及生产。我们为健康而来，\r\n致力于用随意的方式带给您和您家人健康的生活。',
   'prj_href': '/project/details/100006.html'}}}

In [87]:
s_project.find_one({}, projection={'公司名称': 1, '发起人名称': 1}).values()

dict_values(['100010', '广州粤脉文化发展有限公司', '粤脉电商视觉设计'])

## 一、单个项目发起人信息的获取

In [4]:
item = project.find_one({'状态': '众筹成功'}, projection={'公司名称':1, '公司地址':1, '公司工作时间':1, 
                                                        '公司电话':1, '发起人名称':1, '发起人链接':1})

In [5]:
prj_info = project.find_one({'_id':item['_id']}, 
                            projection={'状态':1, '所属类别':1, '项目名称':1, '众筹期限':1, '目标金额':1, '_id':0})

In [6]:
prj_info

{'状态': '众筹成功',
 '众筹期限': 45,
 '所属类别': '健身装备',
 '目标金额': 100000,
 '项目名称': '格莱迪斯微电流智能减脂健腹仪'}

- Head信息

In [7]:
User_Agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.13; rv:60.0) Gecko/20100101 Firefox/60.0'
Host = 'z.jd.com'

- 获取网页内容

In [161]:
def crawData(href, _id):
    headers = {'Host': Host, 
               'User-Agent': User_Agent, 
               'Referer': f'http://z.jd.com/funderCenter.action?flag=2&id={_id}'}
    r = requests.get(href + 'flag=2&id=' + _id)
        
    return r.text

- 解析获取数据

In [9]:
def soupData(raw):
    h_soup = BeautifulSoup(raw, 'html.parser')
    cat_div = h_soup.find('div', {'class': 'tab_cont db'})
    p_d = re.compile('(\d+)')
    
    c_dict = {}
    if cat_div:
        for x in cat_div.findAll('li'):
            prj_href = str(x).split("'")[1]
            prj_id = p_d.findall(prj_href)[0]
            c_dict[prj_id] = {'prj_name': x.p.string, 'prj_desc': x.a.string, 'prj_href': prj_href}
        
    return c_dict

In [200]:
item = {'_id': '107555'}
hrefs = {'支持项目': 'http://z.jd.com/f/my_support.action?',
         '关注项目': 'http://z.jd.com/f/my_focus.action?', 
         '发起项目': 'http://z.jd.com/f/my_project.action?'}

for key in hrefs:
    print(key)
    raw = crawData(hrefs[key], item['_id'])
    c_dict = soupData(raw)
    item[key] = c_dict

支持项目
关注项目
发起项目


In [201]:
item

{'_id': '107555',
 '支持项目': {'85305': {'prj_name': '「华夏匠心」木纹釉惊世之作',
   'prj_desc': '徐志伟大师手造，灵感来源于感慨时光岁月，春去秋来。意在传承传统文化，推广中国传统手工艺制品。',
   'prj_href': '/project/details/85305.html'},
  '84512': {'prj_name': '传统手工铸铁壶大师升华之作',
   'prj_desc': '匠人团队精工新作，传统失蜡法结合传统鎏金技艺打造“佛手托莲”铸铁壶，让您品味专属于您的壶韵。',
   'prj_href': '/project/details/84512.html'}},
 '关注项目': {},
 '发起项目': {'118120': {'prj_name': '华匠左藏·蟹将军铁壶石繁手作',
   'prj_desc': '六个月研发、三十版雕刻方案更迭、半分之十五浇筑成品率，废掉近千个手雕蜡模，只为一个国壶的夏天，蟹将军。支持国壶，让我们一起纵横四海。',
   'prj_href': '/project/details/118120.html'},
  '117100': {'prj_name': '华匠左藏·大雨游龙典藏砂铁壶',
   'prj_desc': '华匠左藏联合关西铁瓶精工制作，大雨游龙，典藏砂铁壶，百年一壶，百味人生，以一把真正的好壶慰藉人生！',
   'prj_href': '/project/details/117100.html'},
  '113654': {'prj_name': '华匠左藏·绽金千朵莲禅意铁壶',
   'prj_desc': '华匠经典托莲铁壶原班人马，臻高设计，巅峰重铸，禅意·绽金千朵莲原铁铁壶，正式上线！石繁大师历两年打磨，颠覆国壶设计，关西铁瓶制作方精工铸造，重磅回归！',
   'prj_href': '/project/details/113654.html'},
  '110901': {'prj_name': '华匠左藏·灰釉跳刀青瓷主人杯',
   'prj_desc': '廖建军手作，青瓷设计金奖作品，灰釉跳刀青瓷主人杯。最具代表性的龙泉青瓷烧制技艺、精湛的跳刀纹技艺与现代创新设计理念相碰撞，唯愿以一只杯

## 二、所有项目发起人*关注的项目、发起的项目和支持的项目*

- 项目成功信息的id及对应公司信息

In [91]:
success_ids = list([x['_id'] for x in s_project.find({'状态': "项目成功"}, projection={"_id": 1})])  # 项目成功信息ids

In [92]:
success_infos = [list(s_project.find_one({"_id": _id}, projection={"_id": 1, '公司名称': 1, '发起人名称': 1}).values())
                     for _id in success_ids]  # 项目成功信息ids

- 失败项目的id及对应公司信息

In [66]:
fail_ids = list(set(x['_id'] for x in f_project.find({}, projection={'_id':1}) if not isinstance(x['_id'], ObjectId)))  # 失败项目信息ids

In [94]:
fail_infos = [list(f_project.find_one({"_id": _id}, projection={"_id": 1, '公司名称': 1, '发起人名称': 1}).values())
                     for _id in fail_ids]

In [69]:
fail_objectids = list(set(x['_id'] for x in f_project.find({}, projection={'_id':1}) if isinstance(x['_id'], ObjectId)))  # _id为ObjectId

In [74]:
trans_fail_ids = []
for _id in fail_objectids:
    x = f_project.find_one({'_id': _id}, projection={'_id':1, '详细信息._id':1})
    trans_fail_ids.append(x['详细信息']['_id'])

In [95]:
for _id in fail_objectids:
    x = f_project.find_one({'_id': _id}, projection={'_id':1, '详细信息._id':1, '详细信息.公司名称': 1, '详细信息.发起人名称': 1})
    fail_infos.append([x['详细信息']['_id'], x['详细信息']['公司名称'], x['详细信息']['发起人名称']])

In [102]:
all_infos = success_infos + fail_infos

In [105]:
company_project_dict = {}
for x in all_infos:
    if x[1] in company_project_dict:
        company_project_dict[x[1]].append(x[0])
    else:
        company_project_dict[x[1]] = [x[0]]

In [108]:
for key in company_project_dict:
    company_project_dict[key] = list(set(company_project_dict[key]))

In [117]:
len(company_project_dict)

4836

In [76]:
ids = success_ids + fail_ids + trans_fail_ids

In [116]:
company_project_dict['贵州山珍宝绿色科技开发有限公司']

['115263', '120294']

In [111]:
fail_infos[:10]

[['1823378321', '龙州香茗茶业有限责任公司', '广西龙州红江橙'],
 ['112470', '昆山丝倍奇纸业有限公司', 'sbile'],
 ['1825084992', '苏州德博电子有限公司', '德克赛弗'],
 ['121475', '贵州荣和烧坊酒业有限公司', '荣和烧坊酒'],
 ['108738', '广州世格汽车用品有限公司', 'ushige'],
 ['120843', '深圳市美华化妆用品有限公司', 'A135*****112'],
 ['120294', '贵州山珍宝绿色科技开发有限公司', 'szbgs168'],
 ['1819086027', '上海画乐宝文化传媒有限公司', '189*****202sf'],
 ['1821338389', '泉州易物宝进出口贸易有限公司', '易物宝进出口贸易'],
 ['112689', '东莞市雅弗莱家具有限公司', 'yafulai']]

## 三、遍历所有项目发起人主页

In [ ]:
item = {'_id': '83904'}
hrefs = {'支持项目': 'http://z.jd.com/f/my_support.action?',
         '关注项目': 'http://z.jd.com/f/my_focus.action?', 
         '发起项目': 'http://z.jd.com/f/my_project.action?'}

for key in hrefs:
    print(key)
    raw = crawData(hrefs[key], item['_id'])
    c_dict = soupData(raw)
    item[key] = c_dict

In [173]:
related_projects = {}
i = 0
for company in company_project_dict:
    try:
        _id = company_project_dict[company][0]
        related_projects[company] = {}
        for key in hrefs:
            raw = crawData(hrefs[key], _id)
            c_dict = soupData(raw)
            related_projects[company][key] = list(c_dict)
        if i % 100 == 0:
            print(related_projects[company])
        i += 1
        time.sleep(0.1)
    except Exception as e:
        print(e)

{'支持项目': [], '关注项目': [], '发起项目': ['101974', '100010', '96471', '95962', '66595', '65944', '65024', '63032']}
{'支持项目': [], '关注项目': [], '发起项目': ['109768', '103666', '103627', '102905', '101201', '100649', '99271', '98354']}
{'支持项目': ['101215'], '关注项目': [], '发起项目': []}
{'支持项目': [], '关注项目': [], '发起项目': ['101890']}
{'支持项目': [], '关注项目': [], '发起项目': ['110798', '107130', '105384', '102592']}
{'支持项目': [], '关注项目': [], '发起项目': ['103389']}
{'支持项目': [], '关注项目': [], '发起项目': ['1845209998', '1845209797', '1845209397', '1837547476', '1836854332', '1836592122', '121189', '120289']}
{'支持项目': [], '关注项目': [], '发起项目': ['107328', '105342']}
{'支持项目': [], '关注项目': [], '发起项目': ['106574']}
{'支持项目': ['120248'], '关注项目': [], '发起项目': ['107804']}
{'支持项目': ['108925', '69873', '64542'], '关注项目': [], '发起项目': ['108925']}
{'支持项目': ['109887'], '关注项目': [], '发起项目': ['109887']}
{'支持项目': ['111203'], '关注项目': [], '发起项目': ['111203']}
{'支持项目': [], '关注项目': [], '发起项目': []}
{'支持项目': [], '关注项目': [], '发起项目': ['113734']}
{'支持项目': [], '关注项

In [176]:
set(list(company_project_dict)) - set(list(related_projects))

set()

In [195]:
for c in company_project_dict:
#     print({**related_projects[c], '在库项目': company_project_dict[c]})
#     break
    db.companies.insert_one({"c_name": c, '在库项目': company_project_dict[c], **related_projects[c]})

In [197]:
db.companies.find_one()

{'_id': ObjectId('6113e5d976430dab487cab13'),
 'c_name': '广州粤脉文化发展有限公司',
 '在库项目': ['101974', '100010'],
 '支持项目': [],
 '关注项目': [],
 '发起项目': ['101974',
  '100010',
  '96471',
  '95962',
  '66595',
  '65944',
  '65024',
  '63032']}

In [198]:
db.companies.count_documents({})

4836

In [188]:
len(company_project_dict)

4836

In [192]:
for k in related_projects:
    print(k, related_projects[k])
    break

广州粤脉文化发展有限公司 {'支持项目': [], '关注项目': [], '发起项目': ['101974', '100010', '96471', '95962', '66595', '65944', '65024', '63032']}


In [193]:
company_project_dict['广州粤脉文化发展有限公司']

['101974', '100010']

In [189]:
len(related_projects)

4836

In [183]:
for k in related_projects:
    try:
        if not related_projects[k]['支持项目'] and not related_projects[k]['关注项目'] and not related_projects[k]['发起项目']:
            print(k)
            _id = company_project_dict[k][0]
            related_projects[k] = {}
            for key in hrefs:
                raw = crawData(hrefs[key], _id)
                c_dict = soupData(raw)
                related_projects[company][key] = list(c_dict)
            time.sleep(0.1)
    except Exception as e:
        print(k, e)
        _id = company_project_dict[k][0]
        related_projects[k] = {}
        for key in hrefs:
            raw = crawData(hrefs[key], _id)
            c_dict = soupData(raw)
            related_projects[company][key] = list(c_dict)
        time.sleep(0.1)

上海宸赞电子有限公司 '支持项目'
中山市同城电子有限公司 '支持项目'
厦门聚弘达电子商务有限公司 '支持项目'
深圳市海仕格科技有限公司 '支持项目'
北京小睿科技有限公司 '支持项目'
西安亲仁康生物技术有限公司 '支持项目'
深圳市舜宝科技有限公司 '支持项目'
上海融黎生物科技有限公司 '支持项目'
深圳市谷云网络科技有限公司 '支持项目'
深圳前海龙兴科技有限公司 '支持项目'
深圳益爽科技有限公司 '支持项目'
深圳诚信汇投资咨询有限公司 '支持项目'
深圳市杰豪实业有限公司 '支持项目'
河南伊茜格特电子科技有限公司 '支持项目'
深圳龙舞科技创新有限公司 '支持项目'
广东云端电子商务有限公司 '支持项目'
深圳市樱桃电子商务有限公司 '支持项目'
深圳市沙漠甘泉科技有限责任公司 '支持项目'
杭州若奇技术有限公司 '支持项目'
深圳市康柏特科技开发有限公司 '支持项目'
成都天艺禾文阿思服饰有限公司 '支持项目'
江苏莎恩科技有限公司 '支持项目'
深圳市云柯达科技有限公司 '支持项目'
广东宝爵新材料实业有限公司 '支持项目'
佑象科技（深圳）有限公司 '支持项目'
深圳市微开科技有限公司 '支持项目'
烟台农禅谷生态农业发展有限公司 '支持项目'
深圳市维特威普科技有限公司 '支持项目'
北京凯玛电子科技有限公司 '支持项目'
深圳市菲莫新创科技有限公司 '支持项目'
深圳市同进共赢科技有限公司 '支持项目'
维恩泰合（北京）科技有限公司 '支持项目'
深圳宜歌一品科技有限公司 '支持项目'
辽宁向上科技有限公司 '支持项目'
深圳市乐享雾芯科技有限公司 '支持项目'
苏州匹利商贸有限公司 '支持项目'
北京黎明星科技有限公司 '支持项目'
贵州省仁怀市正鸿酒业有限公司 '支持项目'
四川狄记酒庄有限公司 '支持项目'
拾趣（北京）科技有限公司 '支持项目'
寰西(广州)贸易有限公司 '支持项目'
北京四海田园农副产品加工有限责任公司 '支持项目'
农品汇国际商业连锁有限公司 '支持项目'
深圳隽和家居用品科技有限公司 '支持项目'
山西汇德电子商务科技有限公司 '支持项目'
佛山市易区热能科技有限公司 '支持项目'
深圳市全息调频医疗器械有限公司 '支持项目'
烟台市金美瑞家具有限公司 '支持项目'
会昌县右水冬菜食品有限公司 '支持项目'

- 插入1个文档

In [16]:
def insert_data(p_id):
    item = {'_id': p_id}
    try:
        for key in hrefs:
            raw = crawData(hrefs[key], item['_id'])
            c_dict = soupData(raw)
            item[key] = c_dict
        p_founder.insert_one(item)
        return 1
    except Exception as e:
        print(f"{p_id}插入失败!错误提示:{e}")
        return 0

- 插入多个文档

In [17]:
def insert_dataset(p_ids):
    inserted, notInserted = [], []
    for p_id in p_ids:
        if insert_data(p_id):
            inserted.append(p_id)
        else:
            notInserted.append(p_id)
            
    return inserted, notInserted

- 计算新生成的id

In [18]:
def get_newIDs(p_founder=p_founder):
    now_ids = []
    new_res = []
    for x in p_founder.find({}):
        temp = list(x['支持项目'].keys()) + list(x['关注项目'].keys()) + list(x['发起项目'].keys())
        now_ids.append(x['_id'])
        new_res.extend(temp)
        
    return set(new_res) - set(now_ids)

- 循环爬取数据

In [19]:
j = 1
while True:
    time.sleep(10)
    print(f"第{j}轮")
    newIDs = get_newIDs()
    if newIDs:
        inserted, notinserted = insert_dataset(newIDs)
        j += 1
    else:
        break

第1轮


In [315]:
p_founder.count_documents({})

6413